<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/single_cls_sobert_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [2]:
!pip install evaluate accelerate

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import wandb
wandb.login(key="ee23d2708eaedb89d7020a87e3ac4dea2b742261")
wandb.init(project="Single label classification", entity="ai_expert", name="sobert_base")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ebinna-lee (ebinna-lee-x). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ebinna-lee (ai_expert). Use `wandb login --relogin` to force relogin


In [5]:
from datasets import load_dataset

dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_train_dataset_10000_title.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_val_dataset_1000_title.csv',
    split='train'
)
dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/stackexchage/stackexchange_test_dataset_1000_title.csv',
    split='train'
)

In [6]:
classes = ['biology', 'cooking', 'diy', 'travel', 'stackoverflow']
class2id = {'biology' :0, 'cooking' : 1, 'diy' : 2, 'travel' : 3, 'stackoverflow' : 4}
id2class = {0 : 'biology', 1: 'cooking', 2 : 'diy', 3 : 'travel', 4 : 'stackoverflow'}

In [7]:
from transformers import AutoTokenizer

model_path = 'mmukh/SOBertBase'
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
import ast

def preprocess_function(example):
   samples = tokenizer(example['title_content'], truncation=True, max_length=2048)
   return samples


In [9]:
tokenized_train_dataset = dataset_train.map(preprocess_function)
tokenized_valid_dataset = dataset_valid.map(preprocess_function)
tokenized_test_dataset = dataset_test.map(preprocess_function)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["title_content"])
tokenized_valid_dataset = tokenized_valid_dataset.remove_columns(["title_content"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["title_content"])

In [11]:
from transformers import DataCollatorWithPadding
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
import evaluate
import numpy as np

In [13]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "micro_precision": precision.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "macro_precision": precision.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "weighted_precision": precision.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
   model_path, num_labels=len(classes),
   id2label=id2class, label2id=class2id)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at mmukh/SOBertBase and are newly initialized: ['bert.embeddings.token_type_embeddings.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(50048, 768, padding_idx=0)
      (position_embeddings): Embedding(2048, 768)
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

In [15]:
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=5,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   metric_for_best_model="accuracy",
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_valid_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Micro Precision,Micro Recall,Micro F1,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1
1,0.490400,0.433587,0.856000,0.856000,0.856000,0.856000,0.857723,0.856000,0.855986,0.857723,0.856000,0.855986
2,0.306800,0.385696,0.887000,0.887000,0.887000,0.887000,0.889690,0.887000,0.887280,0.889690,0.887000,0.887280
3,0.239200,0.450995,0.888000,0.888000,0.888000,0.888000,0.890922,0.888000,0.888416,0.890922,0.888000,0.888416
4,0.131500,0.506067,0.887000,0.887000,0.887000,0.887000,0.889665,0.887000,0.887480,0.889665,0.887000,0.887480
5,0.106700,0.496844,0.897000,0.897000,0.897000,0.897000,0.897167,0.897000,0.897047,0.897167,0.897000,0.897047


TrainOutput(global_step=6250, training_loss=0.28882949798583984, metrics={'train_runtime': 300.135, 'train_samples_per_second': 166.592, 'train_steps_per_second': 20.824, 'total_flos': 582070777003056.0, 'train_loss': 0.28882949798583984, 'epoch': 5.0})

In [17]:
trainer.evaluate()

{'eval_loss': 0.4968443810939789,
 'eval_accuracy': 0.897,
 'eval_micro_precision': 0.897,
 'eval_micro_recall': 0.897,
 'eval_micro_f1': 0.897,
 'eval_macro_precision': 0.8971666598575533,
 'eval_macro_recall': 0.8969999999999999,
 'eval_macro_f1': 0.8970469468851731,
 'eval_weighted_precision': 0.8971666598575534,
 'eval_weighted_recall': 0.897,
 'eval_weighted_f1': 0.8970469468851731,
 'eval_runtime': 1.5951,
 'eval_samples_per_second': 626.903,
 'eval_steps_per_second': 78.363,
 'epoch': 5.0}

In [18]:
trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_train_dataset,
   eval_dataset=tokenized_test_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [19]:
trainer.evaluate()

{'eval_loss': 0.5079270005226135,
 'eval_model_preparation_time': 0.0058,
 'eval_accuracy': 0.899,
 'eval_micro_precision': 0.899,
 'eval_micro_recall': 0.899,
 'eval_micro_f1': 0.899,
 'eval_macro_precision': 0.899247715933134,
 'eval_macro_recall': 0.899,
 'eval_macro_f1': 0.8990819615665842,
 'eval_weighted_precision': 0.899247715933134,
 'eval_weighted_recall': 0.899,
 'eval_weighted_f1': 0.8990819615665839,
 'eval_runtime': 1.5765,
 'eval_samples_per_second': 634.322,
 'eval_steps_per_second': 79.29}

In [20]:
tuned_model_path = '/content/drive/My Drive/AiExpertCource/project/final/single_cls_sobert_base'

In [21]:
model.save_pretrained(tuned_model_path)
tokenizer.save_pretrained(tuned_model_path)

('/content/drive/My Drive/AiExpertCource/project/final/single_cls_sobert_base/tokenizer_config.json',
 '/content/drive/My Drive/AiExpertCource/project/final/single_cls_sobert_base/special_tokens_map.json',
 '/content/drive/My Drive/AiExpertCource/project/final/single_cls_sobert_base/tokenizer.json')

In [22]:
tuned_tokenizer = AutoTokenizer.from_pretrained(tuned_model_path)
tuned_model = AutoModelForSequenceClassification.from_pretrained(tuned_model_path)

In [23]:
# 예측 함수 정의
def predict(texts):
    # 텍스트를 토큰화하고 텐서로 변환
    inputs = tuned_tokenizer(texts, padding='max_length', truncation=True, max_length=2048, return_tensors='pt')

    # 모델을 사용해 예측 수행
    with torch.no_grad():
        outputs = tuned_model(**inputs)
        logits = outputs.logits

    # 시그모이드를 사용해 확률로 변환
    probabilities = torch.sigmoid(logits).numpy()

    predictions = np.argmax(probabilities, axis=1)

    return predictions, probabilities

In [24]:
# 샘플 텍스트 입력 및 예측 수행
sample_texts = [
  "FFmpeg hevc rtsp stream decoding with frame loss" +
  "I need to decode my rtsp stream. When i`m using default HEVC and have some corrupted frames, my screen looks like this: Corrupted frame using HEVC, Grey image. But instead of this, I want to have a corrupted picture with pixels issue, like this: Same corrupted frame, but using HEVC_QSV."
]

# 예측 수행
predictions, probabilities = predict(sample_texts)

In [25]:
print(id2class[predictions[0]])

stackoverflow


In [26]:
wandb.finish()

eval/accuracy,▁▆▆▆███
eval/loss,▄▁▅█▇▇█
eval/macro_f1,▁▆▆▆███
eval/macro_precision,▁▆▇▆███
eval/macro_recall,▁▆▆▆███
eval/micro_f1,▁▆▆▆███
eval/micro_precision,▁▆▆▆███
eval/micro_recall,▁▆▆▆███
eval/model_preparation_time,▁
eval/runtime,▄▁▁▆█▄▁
eval/samples_per_second,▅██▃▁▅█
